In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

KeyboardInterrupt: 

## 共通関数定義

In [ ]:
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}




def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=1000
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [ ]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成

In [ ]:
### application
df = FeatureEngineering2.application()
df.head()

In [ ]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

In [ ]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

In [ ]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

In [ ]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

In [ ]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

In [ ]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

In [ ]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

In [ ]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

## 学習

### 001 baseline

In [ ]:
# split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

In [ ]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

In [ ]:
app_train = app_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
app_test.head()

In [ ]:
#pred_train_1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_1a.csv')
#pred_test_1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1a.csv')
#pred_test_1b.head()

In [ ]:
#app_train = app_train.merge(pred_train_1a, how='left', on='SK_ID_CURR')
#app_test = app_test.merge(pred_test_1a, how='left', on='SK_ID_CURR')
#app_test.head()

In [ ]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

In [ ]:
app_train = app_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
app_test.head()

In [ ]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

In [ ]:
app_train = app_train.merge(train_set3a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3a, how='left', on='SK_ID_CURR')
app_test.head()

In [ ]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

In [ ]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

### 正規化＋対数変換

In [ ]:
app_train_ord = app_train.copy()
app_test_ord = app_test.copy()
app_test_ord.head()

In [ ]:
# infをnanへ置換
app_train_ord = app_train_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord = app_test_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord.head()

In [ ]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(app_test_ord.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [ ]:
### 欠損値補完
#app_train_ord = app_train_ord.fillna(app_train_ord.max() + (app_train_ord.max() * 0.4))
app_train_ord.head()

In [ ]:
### 欠損値補完
#app_test_ord = app_test_ord.fillna(app_test_ord.max() + (app_test_ord.max() * 0.4))
app_test_ord.head()

In [ ]:
### Yao-Jhonson変換
#for column in encoding_columns:
#    modules.process_yj_convert(app_train_ord, app_test_ord, column, [column])#
#
#app_test_ord.head()

In [ ]:
### indexを設定
app_train_ord = app_train_ord.set_index('SK_ID_CURR')
app_test_ord = app_test_ord.set_index('SK_ID_CURR')
app_test_ord.head()

In [ ]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train_ord, app_test_ord, num_cols)
        
app_test_ord.head()

In [ ]:
### indexを削除
app_train_ord = app_train_ord.reset_index()
app_test_ord = app_test_ord.reset_index()
app_test_ord.head()

In [ ]:
x_train = app_train_ord.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train_ord['TARGET']
id_train = app_train_ord[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [ ]:
# データセット作成
x_test = app_test_ord.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test_ord[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [ ]:
#del app_train_ord
#del app_test_ord
#gc.collect()

In [ ]:
#for col in app_train_ord.columns:
#    print(col)

In [ ]:
### パラメータ
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 2000
}


### 学習モデル構築
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

In [ ]:
imp.sort_values("imp", ascending=False)[:20]

In [ ]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

In [ ]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSource2-2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

## パラメータチューニング

In [ ]:
#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [ ]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [ ]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [ ]:
#train_oof, imp, metrics = baseline.train_lgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [ ]:
#app_test_ord

In [ ]:
### 推論用データセットの作成
#x_test = app_test_ord.drop(columns=['SK_ID_CURR','TARGET'])
#id_test = app_test_ord[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

### predict
#test_pred = baseline.predict_lgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

### make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuningLgb.csv", index=None)

In [ ]:
#del app_train_ord
#del app_test_ord
#gc.collect()